# Diplomatura de Especialización en Desarrollo de Aplicaciones con Inteligencia Artificial - Inteligencia Artificial para Juegos (Game AI) - Sesión 4 (Enunciado)

<font color='orange'>Entorno de *k en raya* (**Connect4**, para un caso particular) para búsqueda adversarial en un juego de suma constante.</font>

El presente notebook aborda el problema de busqueda adversarial en el juego de Connect4, que es un caso particular del k-en-raya (este, a su vez, es una generalizacón del 3 en raya pero en tableros de $h$ filas por $v$ columnas y gana el jugador que haga primero una raya de tamaño $k$, uniendo $k$ piezas adyacentes en forma horizontal, vertical o diagonal).

<img src='https://pepeganga.vteximg.com.br/arquivos/ids/202622-1045-1100/78703701.jpg?v=636063347230630000' width=300px>

La mayor parte del entorno de juego (clase TicTacToe) esta implementada, solo esta faltando implementar la funcion que convierte una instancia del TicTacToe al juego de ConnectFour (clase <b>ConnectFour</b>). Estan implementados tambien los algoritmos MIN-MAX  y ALPHA-BETA que pueden decidir movidas en el juego implementado.

Recordar que en el algoritmo `minimax`, los valores altos de utilidad son buenos para el jugador MAX.

Considerar que las fichas del jugador MAX son **X**, mientas que las fichas del jugador MIN son **O**.

### Clase <b>Game</b>

Esta es una clase genérica para definir un entorno de juego. Es parecida a la clase `Problem` de búsqueda, pero en vez del método que devuelve el costo de camino se tiene un metodo que devuelve la utilidad de un jugador en un estado dado. También la funcion test de objetivo es reemplazada por un test de estado terminal (`terminal_test`). Para crear una clase de un juego específico se debe hacer una subclase de `Game` e implementar los métodos actions, `result`, `utility`, y `terminal_test`. El atributo `.initial` (estado inicial del juego) deberá ser inicializado en el constructor de la clase concreta. No editar esta clase `Game`.

In [0]:
class Game:

    def actions(self, state):
        """Retorna una lista de movidas permitidas en el estado actual state."""
        raise NotImplementedError

    def result(self, state, move):
        """Retorna el nuevo estado que resulta de hacer una movida move en el estado state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Retorna el valor de utilidad para el jugador player en el estado terminal state."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Retorna True si el estado state es un estado terminal del juego."""
        return not self.actions(state)

    def to_move(self, state):
        """Retorna el jugador que le toca jugar en el presente estado state."""
        return state.to_move

    def display(self, state):
        """Imprime o displaya el state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players, verbose):
        """Controlador del juego:
        Llama alternadamente a cada jugador pasandole el estado actual del juego y ejecutando la movida retornada."""
        state = self.initial
        numJugada = 0
        while True:
            for player in players:
                move = player(self, state)
                mark_now = self.to_move(state)
                state = self.result(state, move)
                numJugada = numJugada + 1
                if verbose:
                  print("Jugada", numJugada, ": Turno del jugador", player.__name__, "(",mark_now,")")
                  self.display(state)
                  print("*************************************************")
                if self.terminal_test(state):
                    print("Jugada", numJugada, "(final): Turno del jugador", player.__name__, "(",mark_now,")")
                    self.display(state)
                    print("La utilidad del primer jugador (",self.to_move(self.initial),") fue: ")
                    #retorna utilidad del 1er jugador al acabar el juego
                    return self.utility(state, self.to_move(self.initial))

### Clase <b>TicTacToe</b>

Esta es una subclase de `Game` para definir el entorno del juego k en Raya (generalizacion de `TicTacToe`). Las dimensiones del tablero son definidas en el constructor (usando los argumentos: h=número de filas, v=número de columnas, k=número de elementos en raya para ganar). Primer jugador (Max) es 'X' y el otro jugador (Min) es 'O'. Un estado en este juego es una tupla (`GameState`) con los siguientes campos:
 - to_move: almacena el jugador que le toca jugar 
 - utility: almacena la utilidad del estado
 - board: almacena las posiciones ocupadas en el tablero en la forma de un dicccionario de entradas {(x, y): Player}, donde Player puede ser 'X' o 'O'
 - moves: almacena las movidas posibles a partir del estado en la forma de una lista de tuplas que representan posiciones (x, y) 

In [0]:
from collections import namedtuple
GameState = namedtuple('GameState', 'to_move, utility, board, moves') #Un estado es una tupla con nombres de campos (namedtuple)
import random
import itertools
import copy

def k_in_row(board, player, square, k): #Adaptado de https://github.com/aimacode/aima-python
    """Devuelve True, si el jugador con marca "player" tiene k piezas en una fila, desde la posición square."""
    def in_row(x, y, dx, dy): return 0 if board.get((x, y)) != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

class TicTacToe(Game):
    
    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k

        assert k>2, "k debe ser mayor que dos!"
        assert k<=h, "k debe ser menor o igual que h"
        assert k<=v, "k debe ser menor o igual que v"

        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """Movidas legales son todas las posiciones aun sin marcar (el estado almacena las movidas legales)"""
        return state.moves

    def result(self, state, move):
        """Retorna el nuevo estado de hacer la movida move en el estado state ."""
        if move not in state.moves:
            return state  # Si es una movida ilegal retorna sin cambiar el estado
        board = state.board.copy()
        board[move] = state.to_move

        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Retorna la utilidad del player en estado terminal state; 1 si ganó, -1 si perdió, 0 empate."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """Un estado es terminal si hay un ganador o no hay mas movidas posibles."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """Retorna  1 si player='X'  ha llegado a estado terminal ganador con movida move, 
           Retorna -1 si player='O' ha llegado a estado terminal ganador con movida move; 
           Retornas 0 en cualquier otro caso"""
        #Cálculo de utilidad para el caso general (k>1)
        win = k_in_row(board, player, move, self.k)
        return 0 if not win else +1 if player == 'X' else -1
        

### Algoritmo  <b>MIN-MAX</b>

Este algoritmo escoge una movida para el jugador de turno en un juego dado (game). El algoritmo obtiene recursivamente los valores minimax de los estados sucesores buscando en profundidad en el arbol de juego los estados terminales. De estos estados toma su valor de utilidad para calcular la utilidad de los padres y asi sucesivamente hasta tener la utilidad de todos los sucesores del estado inicial para decidir la movida a ejecutar. 
La implementacion de esta busqueda es a traves de una recursion alternada de las funciones max_value y min_value (una llama a la otra) hasta alcanzar un estado terminal. Cuando la recursion termina todas las movidas tienen una utilidad y se escoje la movida de mayor valor.


In [0]:
argmax = max
infinity = float('inf')

def minimax_decision(state, game):

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minimax_decision:
    #Cuando la recursion termina todas las movidas tienen una
    #utilidad y se escoje la movida de mayor valor.
    return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))

### Algoritmo  <b>ALPHA-BETA</b>

Este algoritmo escoge una movida para el jugador de turno en el juego, evitando explorar las ramas que no son relevantes para tomar una decisión de movida en el estado actual. Alpha-Beta hace uso de las funciones max_value y min_value de MIN-MAX pero utilizando dos variables: alpha y betha. La variable apha mantiene la mejor opcion (la más alta utilidad) encontrada para MAX a lo largo del camino. La variable betha mantiene la mejor opcion (la más baja utilidad) encontrada para MIN. A medida que el algoritmo avanza se va actualizando alpha y betha y se poda un nodo cuando el valor del nodo es menor que el valor alpha (para MAX) o mayor que el valor betha (para MIN).

In [0]:
def alphabeta_search(state, game):
  
    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_search:
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


### Algoritmo  <b>EXPECTIMINIMAX</b>
Fuente del Pseudocódigo: CS188 - UC Berkeley


<img src='https://raw.githubusercontent.com/gracefulPotato/connect-four-minimax-expectimax/master/expectimax.png' width=500px>

In [0]:
argmax = max
infinity = float('inf')

def expectiminimax_search(state, game, probaUniforme = True):

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, exp_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    def exp_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = 0
        listaAcciones = game.actions(state)
        if(probaUniforme):
          for a in listaAcciones:
            p = 1/(len(listaAcciones)) #la misma probabilidad para todas las acciones
            v = v + p * max_value(game.result(state, a))
        else:

          def probabilidad(state, a, listaAcciones):
            raise NotImplementedError

          for a in listaAcciones:
            p = probabilidad(state,a,listaAcciones) #la probabilidad es de acuerdo a la acción
            v = v + p*max_value(game.result(state, a))
  
        return v

    # Body of expectiminimax_decision:
    return argmax(game.actions(state),
                  key=lambda a: exp_value(game.result(state, a)))

### Jugadores </b>

A seguir se implementan 4 agentes jugadores que pueden hacer movidas en un entorno de juego, dado su estado :
- <b>minimax_player</b>:   jugador que hace movidas de acuerdo al algoritmo MIN-MAX
- <b>alphabeta_player</b>: jugador que hace movidas de acuerdo al algoritmo ALPHA-BETA
- <b>expectimax_player</b>: jugador que hace movidas de acuerdo al algoritmo EXPECTIMINIMAX
- <b>random_player</b>:    jugador que hace movidas aleatorias (es facil ganarle :v )
- <b>human_player</b>:     solicita la movida a un humano


In [0]:
def minimax_player(game, state):
    return minimax_decision(state, game)

def alphabeta_player(game, state):
    return alphabeta_search(state, game)

def expectimax_player(game, state):
    return expectiminimax_search(state, game)    

def random_player(game, state):
    return random.choice(game.actions(state))

def human_player(game, state):
    print("Estado actual:")
    game.display(state)
    print("Movidas disponibles: {}".format(game.actions(state)))
    print("")
    move_string = input('Cuál es tu movida? ')
    try:
        move = eval(move_string)
    except NameError:
        move = move_string
    return move

### Connect Four </b>

Connect Four es una variante del juego TicTacToe, en la cual el tablero es una grilla de 6x7 casillas en total (en la versión comercial del juego de mesa). Además, se agrega la restriccion de que para cada columna, solo está permitido agregar fichas en la casilla vacía de menor altura (físicamente, como si las fichas fueran atraídas hacia abajo por la gravedad). Por velocidad de cómputo, se eligen los valores de h=4, v=4 y k=3.

In [0]:
class ConnectFour(TicTacToe):
    def __init__(self): super().__init__(h=4, v=4, k=3)#(h=6, v=7, k=4)

    def actions(self, state):
        """In each column you can play only the lowest empty square in the column."""
        #COMPLETAR CÓDIGO
        raise NotImplementedError
        return ?

In [0]:
cFour = ConnectFour()
print(cFour.play_game(alphabeta_player, alphabeta_player, verbose=True))

Jugada 1 : Turno del jugador alphabeta_player ( X )
. . . . 
. . . . 
. . . . 
X . . . 
*************************************************
Jugada 2 : Turno del jugador alphabeta_player ( O )
. . . . 
. . . . 
O . . . 
X . . . 
*************************************************
Jugada 3 : Turno del jugador alphabeta_player ( X )
. . . . 
X . . . 
O . . . 
X . . . 
*************************************************
Jugada 4 : Turno del jugador alphabeta_player ( O )
O . . . 
X . . . 
O . . . 
X . . . 
*************************************************
Jugada 5 : Turno del jugador alphabeta_player ( X )
O . . . 
X . . . 
O . . . 
X X . . 
*************************************************
Jugada 6 : Turno del jugador alphabeta_player ( O )
O . . . 
X . . . 
O O . . 
X X . . 
*************************************************
Jugada 7 : Turno del jugador alphabeta_player ( X )
O . . . 
X X . . 
O O . . 
X X . . 
*************************************************
Jugada 8 : Turno del jugador alpha